# Homework 6: NumPy(29 pts)

name: Talia Duffy

email: taliagd@umich.edu

This homework assignment took me 6 hours in total to complete. (Please help us to gauge the difficulty of the assignment.)

## Collaboration Disclosure

In the cell below, please list *everyone* with whom you discussed any of the homework problems, excluding only the GSIs and the course instructor. 

If you did not discuss the homework with anyone else, write __"I did not discuss this homework with anyone."__

Even if you discuss questions with other, the code you submit must be only yours. All work is checked with the [MOSS plagiarism detector](https://theory.stanford.edu/~aiken/moss/).

I did not discuss this homework with anyone.

---

## Submission Instructions
Your homework solutions should be written entirely in this Jupyter notebook file. Once it contains your solutions, you should submit this notebook through Canvas. 


Before submitting, please make sure to __Cells->Run All__ executes without errors; errors in your code translate directly to point deductions. 
In general, you don't need to do explicitly raise errors (e.g. with the ```raise``` function) if we don't ask you to in the problem statement.
However, even in cases where we ask you to check for errors, your submission should not contain any examples of your functions actually raising those errors.

Note that many parts of this homework where you are expected to type in code will have ```NotImplementedError()``` as a placeholder. You need to delete this function and replace it with your own code.

## Homework tips 

1. **Start early!** If you run into trouble installing things or importing packages, it’s
best to find those problems well in advance, not the night before your assignment is
due when we cannot help you!

2. **Make sure you back up your work!** At a minimum, do your work in a Dropbox
folder. Better yet, use git, which is well worth your time and effort to learn.

3. **Be careful to follow directions!** Remember that Python is case sensitive. If
you are ask you to define a function called my_function and you define a function
called My_Function, you will not receive full credit. You may want to copy-paste
the function names below to make sure that the functions in your notebook match.

## Error checking

You do not need to do error checking (raising errors, etc.) in your code unless we explicitly ask you to so in a problem.


## Nbgrader

We will be using `nbgrader` to grade your jupyter notebook. You will notice some `read-only` cells in the assignment that contain `assert` statements. These are tests that your code must pass for your solution to be correct. If any of the tests fail, you will get an python error and not get points for that question. 

**Note:** The tests shown not are not comprehensive; additional tests will be used at grading time. You are encouraged to read the problem carefully and verify your code covers all possible cases.

**Be careful:** If a jupyter notebook cell takes longer than `60s` to run, the autograder will not grade it and you will receive zero credit for that question.

## Question 1:. The Bakersville Bake-off! (3 pts)

The houses in the town of Bakersville are arranged on a grid with R rows (numbered 1 through R) and C columns (numbered 1 through C). The house located on row r and on column c is denoted by (r, c). We say two houses are **neighbors** if they have a common side. 

During the annual Bakersville Bake-off, each household bakes a certain number of cakes to share with their neighbors.
The town comptroller keeps track in an $R \times C$ matrix $A$, where $A_{r, c}$ is the number of cakes which household $(r, c)$ has baked.

The stability of the social order in Bakersville depends on each house baking enough cakes so they can give at least one cake to each neighbor and submit one cake to be judged. Thus, we say a house is **stable** if the number of cakes $A_{r, c}$ is strictly greater than its number of neigbors. We say Bakersville is **stable** if every house is stable. 

The comptroller has asked you to define a function `is_stable` that takes an $R \times C$ numpy array (i.e., an ndarray) as its only argument and returns a Python Boolean that is True if and only if the argument is a stable array and False otherwise. 

In [21]:
import numpy as np

def is_stable(ville):

    if ville.ndim == 1:
        (r,c) = 1, ville.shape[0]
    else: 
        (r,c) = ville.shape[0], ville.shape[1]

    neighbors = np.zeros_like(ville)

    first_r = 0
    last_r = max(0, r-1)
    first_c = 0
    last_c = max(0, c-1)

    for i in range(r):
        for j in range(c):
            
            if i in (first_r,last_r) and j in (first_c,last_c): 
                if ville.ndim == 1: neighbors[j] += 1
                elif c == 1: neighbors[i] += 1
                else: neighbors[i,j] += 2 

            elif i in (first_r,last_r) or j in (first_c,last_c):
                if ville.ndim == 1: neighbors[j] += 2
                elif c == 1: neighbors[i] += 2
                else: neighbors[i,j] += 3
            else:
                neighbors[i,j] += 4

    stability = ville > neighbors

    return bool(stability.all())


In [22]:
array1 = np.array([4, 7, 8,
                   5, 10, 9,
                   3, 5, 7]).reshape(3, 3)
array2 = np.array([3, 2, 4,
                   1, 1, 2,
                   3, 3, 3]).reshape(3, 3)


assert is_stable(array1)
assert not is_stable(array2)
assert not is_stable(np.array([1., 0., 1.]))

## Question 2: Down with `for` loops (12 pts)
Each of the problems below contains a function that uses `for` loops to perform a certain operation on arrays. Your job is to re-write these function using only Numpy array manipulations and library functions (e.g. `np.xxx()`). Do not use any `for` or `while` loops, iterators, generators, or list comprehensions in your solutions.

**2(a)**  Return all of the rows of the 2-D integer matrix `A` where where each entry of the row is distinct (3 pts)

In [23]:
def distinct_rows_py(A):
    'Return all rows of A that have completely distinct entries.'
    return np.array([a for a in A if len(set(a)) == len(a)])
A = np.eye(5)
print(distinct_rows_py(A))
B = np.arange(9).reshape(3, 3)
print(distinct_rows_py(B))

[]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [24]:
import numpy as np
def distinct_rows_np(A):
    slay = np.apply_along_axis(
        func1d = lambda row: len(set(row)) == len(row),
        axis = 1,
        arr = A
    )
    return A[slay,] 

In [25]:
A = np.eye(5)
np.testing.assert_allclose(distinct_rows_np(A), np.empty([0, 5]))

**2(b)** Given a vector $v$ of length $n$, and an integer $0<k<=n$, return a 2D array as: (3 pts)
```
[[v[0], ..., v[k-1]],
 [v[1], ..., v[k]  ],
 [v[2], ..., v[k+1]],
 [ .     .       . ],
 [ .     .       . ],
 [ .     .       . ],
 [v[n-k+1, ..., v[n]]
 ```

In [26]:
def sliding_stack_py(v, k):
    "Stack sliding windows of v of length k."
    rows = []
    for i in range(len(v) - k + 1):
        rows.append(v[i : (i + k)])
    return np.array(rows)
sliding_stack_py(np.array([1, 2, 3, 4, 5]), 3)

array([[1, 2, 3],
       [2, 3, 4],
       [3, 4, 5]])

In [27]:
import numpy as np
def sliding_stack_np(v, k):
    n = len(v)
    ids = np.arange(0, k) + np.arange(0, n - k + 1).reshape(-1, 1) #broadcasting woo
    result = np.take(v, ids)
    return result

In [28]:
np.testing.assert_allclose(sliding_stack_np(np.array([1, 2, 3, 4, 5]), 3), np.array([[1, 2, 3],[2, 3, 4],[3, 4, 5]]))
np.testing.assert_allclose(sliding_stack_np(np.array([1, 2, 3, 4, 5]), 5), np.array([[1, 2, 3, 4, 5]]))

**2(c)** Given a vector of non-negative integers `v`, with `max(v) = m`, return a vector `c` of length `m + 1` such that `c[i]` is the number of times that the integer `i` appears in `v`.(3 pts)

In [29]:
def digit_count_py(v):
    m = max(v)
    ret = np.zeros(m + 1, int)
    for vv in v:
        ret[vv] += 1
    return ret
v = np.array([0, 0, 1, 1, 2, 2, 2, 3, 5])
digit_count_py(v)

array([2, 2, 3, 1, 0, 1])

In [30]:
import numpy as np
def digit_count_np(v):
    idx = np.arange(0, max(v) + 1).reshape(-1, 1)
    result = np.apply_along_axis(func1d = lambda el: sum(v == el), axis = 1, arr = idx)
    return result

In [31]:
v = np.array([0, 0, 1, 1, 2, 2, 2, 3, 4])
np.testing.assert_allclose(digit_count_np(v), np.array([2, 2, 3, 1, 1]))
v = np.array([0, 0, 0, 0, 2, 2, 2, 3, 4])
np.testing.assert_allclose(digit_count_np(v), np.array([4, 0, 3, 1, 1]))

**2(d)** Call a square $n\times n$ matrix $A$ *countersymmetric* if $A_{ij} = A_{n-j,n-i}$ for all $i$ and $j$. An example of such a matrix is:

$$
\begin{pmatrix}
4 & 3 & 2 & 1 & 0\\
8 & 7 & 6 & 5 & 1\\
11 & 10 & 9 & 6 & 2\\
13 & 12 & 10 & 7 & 3\\
14 & 13 & 11 & 8 & 4
\end{pmatrix}
$$

Write a function `is_countersym` that checks this property: (3 pts)

In [32]:
def is_countersym_py(A):
    "Returns True if A is countersymmetric"
    n = A.shape[0]
    for i in range(n):
        for j in range(n):
            if A[i, j] != A[n - j - 1, n - i - 1]:
                return False
    return True
cs_matrix = np.array([[ 4,  3,  2,  1,  0], [ 8,  7,  6,  5,  1], [11, 10,  9,  6,  2], [13, 12, 10,  7,  3], [14, 13, 11,  8,  4]])
is_countersym_py(cs_matrix)

True

In [33]:
import numpy as np
def is_countersym_np(A):
    if A.ndim == 1:
        return True
    flipped = np.flip(A, axis = 1)
    flipped_transpose = np.linalg.matrix_transpose(flipped)
    return(bool(np.all(flipped == flipped_transpose)))

In [34]:
cs_matrix = np.array([[ 4,  3,  2,  1,  0], [ 8,  7,  6,  5,  1], [11, 10,  9,  6,  2], [13, 12, 10,  7,  3], [14, 13, 11,  8,  4]])
assert is_countersym_np(cs_matrix), "The matrix is not counter-symmetric"


## Question 3: $k$-means clustering (14 pts)

$k$-means is a fundamental algorithm for clustering multivariate data. The inputs to the algorithm are:
- An $n\times p$ data matrix $X$ consisting of $n$ observations of a $p$-dimensional feature vector, and
- A $k\times p$ matrix $C$ containing initial guesses for each $k$ cluster centers.

The algorithm proceeds by iteratively a) assigning each point to the nearest cluster center, and b) recomputing the cluster centers as the mean of all of the currently assigned points. Here is a partial implementation:

In [35]:
def kmeans(X, C):
    """
    K-means algorithm.

    Args:
        - X: ndarray, shape (n, p), n observations of p-dimensional feature vector
        - C: ndarray, shape (k, p), k initial cluster centers

    Returns:
        Tuple of length two:
        The first entry is integer ndarray, shape (n), cluster assignments for each data point
        The second entry is ndarray, shape (k, p), centers of each cluster
    """
    assert X.shape[1] == C.shape[1]  # p should match
    old_assignments = None
    while True:
        assignments = nearest_cluster(X, C)
        if np.all(assignments == old_assignments):
            # converged
            return assignments, C
        old_assignments = assignments
        C = compute_centroids(X, assignments)

You will finish implementing this algorithm by completing the missing functions `nearest_cluster()` and `compute_centroids()` below. Note: as in the Question 1, avoid using loops, iterators, or comprehensions. You can find solutions with only broadcasting

**3(a)** Implement the function `nearest_cluster`. It should take two array arguments, the data points `X` and the cluster centers `C`, and return an integer array giving the index in `C` which is nearest to each point in `X`. (3 pts)

In [36]:
# used for testing 

x = np.array(([1,2,3,4,5,6,7,8,9,10,11,12])).reshape(4,3)
x

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [37]:
c = np.array((0,1,2,10,13,11)).reshape(2,3)
c

array([[ 0,  1,  2],
       [10, 13, 11]])

In [38]:
a = nearest_cluster(x, c)

In [39]:
sorted_x = x[np.argsort(a)]
sorted_x

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [40]:
sorted_a = np.sort(a)

In [41]:
idx = np.flatnonzero(sorted_a[1:] != sorted_a[:-1])
idx

array([1])

In [42]:
cluster_groups = np.array(np.split(sorted_x,idx+1))
cluster_groups

array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]])

In [43]:
cluster_groups.mean(axis = 1)

array([[ 2.5,  3.5,  4.5],
       [ 8.5,  9.5, 10.5]])

In [44]:
# more testing
"""
dists_each_cluster = np.apply_along_axis(
    func1d = lambda cluster: np.apply_along_axis(
        func1d = lambda row: sum((row - cluster) ** 2),
        axis = 1,
        arr = c
    ),
    axis = 1,
    arr = x
)
cluster_assignments = np.argmin(dists_each_cluster, axis = 1)
cluster_assignments
"""

'\ndists_each_cluster = np.apply_along_axis(\n    func1d = lambda cluster: np.apply_along_axis(\n        func1d = lambda row: sum((row - cluster) ** 2),\n        axis = 1,\n        arr = c\n    ),\n    axis = 1,\n    arr = x\n)\ncluster_assignments = np.argmin(dists_each_cluster, axis = 1)\ncluster_assignments\n'

In [45]:
#def old_nearest_cluster(X, C):

"""
    For each point in X, find the nearest point in C.

    Args:
        X: ndarray, shape (n, p), n points of dimension p.
        C: ndarray, shape (k, p), k points of dimension p.

    Returns:
        Integer array of length n, [j[1], j[2], ..., j[n]], such that |X[i] - C[j[i]]| <= |X[i] - C[ell]| for 1 <= ell <= k.
"""
    
#    dists_each_cluster = np.apply_along_axis(
#        func1d = lambda cluster: np.apply_along_axis(
#            func1d = lambda row: sum((row - cluster) ** 2),
#            axis = 1,
#            arr = C
#        ),
#        axis = 1,
#        arr = X
#    )

#    cluster_assignments = np.argmin(dists_each_cluster, axis = 1)
#    return cluster_assignments
    

'\n    For each point in X, find the nearest point in C.\n\n    Args:\n        X: ndarray, shape (n, p), n points of dimension p.\n        C: ndarray, shape (k, p), k points of dimension p.\n\n    Returns:\n        Integer array of length n, [j[1], j[2], ..., j[n]], such that |X[i] - C[j[i]]| <= |X[i] - C[ell]| for 1 <= ell <= k.\n'

In [46]:
#xr = x[:,None,:]
#xr.shape
#cr = c[None,:,:]
#cr.shape

In [47]:
#cluster_dists = np.linalg.norm(xr - cr, axis = -1)

In [48]:
#(xr - cr).shape
#cluster_dists.shape

In [49]:
def nearest_cluster(X, C):
    Xr = X[:, None, :]
    Cr = C[None, :, :]

    cluster_dists = np.linalg.norm(Xr - Cr, axis = -1)
    return np.argmin(cluster_dists, axis = 1)
    

In [50]:
ans = np.array([0, 0, 0, 0, 1, 1, 1, 2, 2, 2])
X = np.array([[-1], [0.5], [1], [1.3], [1.6], [2], [2.3], [2.7], [3], [100]])
C = np.array([[1], [2], [3]])
np.testing.assert_allclose(nearest_cluster(X, C), ans)

X = np.array([[-101, 1], [1,1], [1, 1.01], [1.5, 1.6], [2.5, -3], [2.5, 2.7], [3, 33]])
C = np.array([[1, 1], [2,2], [3,3]])
ans = np.array([0, 0, 0, 1, 0, 2, 2])
np.testing.assert_allclose(nearest_cluster(X, C), ans)

**3(b)** Implement the function `compute_centroids`. It should take two array arguments, the data points `X` and the assignment array `a`, and return an $k \times p$ array containing the cluster centroids (averages) for each point assigned to cluster $0, \dots, k-1$. (You may assume that every entry of $a$ is between $0$ and $k-1$, inclusive.) (3 pts) 

In [51]:
#labels = np.arange(0, max(cluster_assignments)+1)
#labels = labels[:, None]
#x[np.nonzero(cluster_assignments == 1)].mean(axis = 0)
#new_centroids = np.apply_along_axis(
#    func1d = lambda k: x[np.nonzero(cluster_assignments == k)].mean(axis = 0),
#   arr = labels,
#   axis = 1
#)
#new_centroids
#new_centroids.reshape(max(cluster_assignments)+1, -1)

In [52]:
def t_compute_centroids(X, a):
    labels = np.arange(0, max(a)+1)
    labels = labels[:, None]
    new_centroids = np.apply_along_axis(
        func1d = lambda k: X[np.nonzero(a == k)].mean(axis = 0), 
        arr = labels,
        axis = 1
    )
    return new_centroids.reshape(max(a) + 1, -1)

In [53]:
def compute_centroids(X, a):
    sorted_X = X[np.argsort(a)]
    sorted_a = np.sort(a)

    split_idx = np.flatnonzero(sorted_a[:-1] != sorted_a[1:])
    cluster_groups = np.array(np.split(sorted_X, split_idx + 1))
    return cluster_groups.mean(axis = 1)

In [62]:
X[np.argsort(a)]

array([[4.17022005e-01, 7.20324493e-01],
       [3.96767474e-01, 5.38816734e-01],
       [4.19194514e-01, 6.85219500e-01],
       [3.13424178e-01, 6.92322616e-01],
       [1.03226007e-01, 4.47893526e-01],
       [4.91573159e-01, 5.33625451e-02],
       [1.82882773e-02, 7.50144315e-01],
       [1.46755891e-01, 9.23385948e-02],
       [6.86500928e-01, 8.34625672e-01],
       [1.14374817e-04, 3.02332573e-01],
       [2.11628116e-01, 2.65546659e-01],
       [1.93669579e-02, 6.78835533e-01],
       [2.80443992e-01, 7.89279328e-01],
       [9.08595503e-01, 2.93614148e-01],
       [2.73875932e-02, 6.70467510e-01],
       [5.74117605e-01, 1.46728575e-01],
       [1.86260211e-01, 3.45560727e-01],
       [8.00744569e-01, 9.68261576e-01],
       [1.40386939e-01, 1.98101489e-01],
       [1.69830420e-01, 8.78142503e-01],
       [8.76389152e-01, 8.94606664e-01],
       [8.50442114e-02, 3.90547832e-02],
       [4.17304802e-01, 5.58689828e-01],
       [2.04452250e-01, 8.78117436e-01],
       [9.834683

In [63]:
sorted_a = np.sort(a)
sorted_a

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4,
       4, 4, 4, 4, 4, 4, 4, 4])

In [64]:
split_idx = np.flatnonzero(sorted_a[:-1] != sorted_a[1:])
split_idx

array([ 6, 11, 15, 20])

In [71]:
np.split(X, split_idx+1)

[array([[4.17022005e-01, 7.20324493e-01],
        [1.14374817e-04, 3.02332573e-01],
        [1.46755891e-01, 9.23385948e-02],
        [1.86260211e-01, 3.45560727e-01],
        [3.96767474e-01, 5.38816734e-01],
        [4.19194514e-01, 6.85219500e-01],
        [2.04452250e-01, 8.78117436e-01]]),
 array([[0.02738759, 0.67046751],
        [0.4173048 , 0.55868983],
        [0.14038694, 0.19810149],
        [0.80074457, 0.96826158],
        [0.31342418, 0.69232262]]),
 array([[0.87638915, 0.89460666],
        [0.08504421, 0.03905478],
        [0.16983042, 0.8781425 ],
        [0.09834683, 0.42110763]]),
 array([[0.95788953, 0.53316528],
        [0.69187711, 0.31551563],
        [0.68650093, 0.83462567],
        [0.01828828, 0.75014431],
        [0.98886109, 0.74816565]]),
 array([[0.28044399, 0.78927933],
        [0.10322601, 0.44789353],
        [0.9085955 , 0.29361415],
        [0.28777534, 0.13002857],
        [0.01936696, 0.67883553],
        [0.21162812, 0.26554666],
        [0.4915731

In [60]:
import numpy as np
np.random.seed(1)
X = np.random.rand(30, 2)
a = np.random.randint(0, 5, size=30)
assert compute_centroids(X, a).shape == (5, 2)
assert np.allclose(X[a == 0].mean(axis=0), compute_centroids(X, a)[0])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

**3(c)** The performance of the $k$-means algorithm is known to depend heavily on the starting point (the initial clusters `C` passed in as the second argument.) In some cases, using a "good" starting point can dramatically improve the performance of the algorithm.

The $k$-means++ algorithm is designed to find such a good starting point. [According to Wikipedia](https://en.wikipedia.org/wiki/K-means%2B%2B), the steps of $k$-means++ are:

1. Choose one center uniformly at random among the data points.
2. For each data point $x$ not chosen yet, compute $D(x)$, the distance between $x$ and the nearest center that has already been chosen.
3. Choose one new data point at random as a new center, using a weighted probability distribution where a point $x$ is chosen with probability proportional to $D(x)^2$.
4. Repeat Steps 2 and 3 until $k$ centers have been chosen.

Implement this algorithm using the skeleton provided below. As before, your implementation should only use Numpy functions--no additional loops or comprehensions. 

**Note**: To ensure reproducibility, the parts of the algorithm that rely on ranndomness are provided for you. Your job is to fill in the missing lines necessary to complete the algoritm. (5 pts.)

In [ ]:
def kmeanspp(X, k, rng):
    """
    k-means++ algorithm.

    Args:
        - X: ndarray, shape (n, p), as above.
        - k, the number of clusters.
        - rng: instance of np.random.Generator().

    Returns:
        ndarray, shape (k, p), cluster centers.
    """
    n, p = X.shape
    C = np.zeros((k, p))
    # step 1
    j = rng.choice(n)
    C[0] = X[j]
    X_remaining = np.delete(X, obj = j, axis = 0) # we only calculate for points in X not already chosen
    for i in range(1, k):
        # YOUR CODE HERE
        X_remaining_3 = X_remaining[:, None, :] # changing dims for broadcasing
        C_3 = C[None, :i+1, :] # for clusters, only keep rows that have already been filled
            # not-yet-filled zeroes would mess up the distance
        d2 = np.linalg.norm(X_remaining_3 - C_3, axis = -1) ** 2 # squared distance
        w = d2 / sum(d2) # weights
        w = w[:,0] # making weights 1d so it can go in rng.choice
        # step 3
        j = rng.choice(n - i, p=w) # n - i because i points have been deleted in the ith iteration
        C[i] = X_remaining[j]
        X_remaining = np.delete(X_remaining, obj = j, axis = 0) # again remove chosen point
    return C

In [ ]:
import numpy as np

np.random.seed(1)
X = np.random.rand(30, 4)
k = 2
rng = np.random.default_rng(12345)
assert kmeanspp(X, k, rng).shape == (2, 4)
ans = np.array([[0.87638915, 0.89460666, 0.08504421, 0.03905478], 
                [0.6634415 , 0.62169572, 0.11474597, 0.94948926]])
np.testing.assert_allclose(kmeanspp(X, k, rng), ans)

**3(d)** In order to measure how good a clustering is, we can define the *within-class variance* 

$$ V(\mathbf{X}, \mathbf{a}, \mathbf{C}) = \sum_{i=1}^n \| \mathbf{x}_i - \mathbf{c}_{a_i} \|^2,$$

where the $i$-th element of $\mathbf{a}=\{a_1,\dots,a_n\}$ is the cluster assignment of observation $i$, and $\mathbf{C}=(\mathbf{c}_1,\dots,\mathbf{c}_k)$ are the centers of each cluster. Thus, $V(\mathbf{X}, \mathbf{a}, \mathbf{C})$ is the sum of the squared distance from each data point to the center of its assigned cluster.

Implement this function. (Again, no loops, just use Numpy functions.) (3 pts) 

In [ ]:
def V(X, a, C):
    labels = np.arange(0, max(a)+1)
    labels = labels[:, None]
    within_cluster_dists = np.apply_along_axis( # for each cluster, find sum of squared distances
        func1d = lambda k: sum(np.linalg.norm(X[np.nonzero(a == k)] - C[k], axis = 1) ** 2),
        arr = labels,
        axis = 1
    )
    return sum(within_cluster_dists) # return sum over all clusters

In [ ]:
import numpy as np

X = np.arange(9).reshape(3, 3)
a = np.arange(3)
C = 2 + X
np.testing.assert_allclose(V(X, a, C), 36)
